In [1]:
import io
from pathlib import Path

import img2pdf  # type: ignore
import typer

In [2]:
def pil_to_pdf_img2pdf(pil_images, output_path: Path):
    """
    images2pdf
    """
    if not pil_images:
        return

    image_bytes_list = []

    for img in pil_images:
        if img.mode != "RGB":
            img = img.convert("RGB")

        img_buffer = io.BytesIO()
        img.save(img_buffer, format="JPEG", quality=95)
        img_bytes = img_buffer.getvalue()
        image_bytes_list.append(img_bytes)

    try:
        pdf_bytes = img2pdf.convert(image_bytes_list)
        assert pdf_bytes is not None
        with open(output_path, "wb") as f:
            f.write(pdf_bytes)

    except Exception as e:
        print(f"error: {e}")

In [3]:
from paddleocr import PaddleOCRVL

In [5]:
pipeline = PaddleOCRVL(
            use_doc_orientation_classify=False,
            use_doc_unwarping=False,
            use_chart_recognition=False,
        )

/usr/local/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-DocLayoutV2', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/paddleocr/.paddlex/official_models/PP-DocLayoutV2`.
Creating model: ('PaddleOCR-VL-0.9B', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/paddleocr/.paddlex/official_models/PaddleOCR-VL`.
Loading configuration file /home/paddleocr/.paddlex/official_models/PaddleOCR-VL/config.json
Loading weights file /home/paddleocr/.paddlex/official_models/PaddleOCR-VL/model.safetensors
use GQA - num_heads: 16- num_key_value_heads: 2
use GQA - num_heads: 16- num_key_value_hea

In [10]:
pdf_path = Path("input/ch4.pdf")
output_dir = Path("output_circle")

In [12]:
output = pipeline.predict_iter(input=str(pdf_path))

In [13]:
markdown_list = []
markdown_images = []
res_images = []

In [14]:
for res in output:
    md_info = res.markdown
    markdown_list.append(md_info)
    markdown_images.append(md_info.get("markdown_images", {}))
    res_images.append(res.img)

/usr/local/lib/python3.10/site-packages/paddle/tensor/creation.py:1088: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach(), rather than paddle.to_tensor(sourceTensor).
  return tensor(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/site-packages/paddle/utils/decorator_utils.py:420: Warning: 
Non compatible API. Please refer to https://www.paddlepaddle.org.cn/documentation/docs/en/develop/guides/model_convert/convert_from_pytorch/api_difference/torch/torch.max.html first.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Se

In [15]:
markdown_texts = pipeline.concatenate_markdown_pages(markdown_list)
mkd_file_path = output_dir / f"{pdf_path.stem}.md"
mkd_file_path.parent.mkdir(parents=True, exist_ok=True)

with open(mkd_file_path, "w", encoding="utf-8") as f:
    f.write(markdown_texts)

In [21]:
for layout in res_images[0].keys():
    print(layout)

layout_det_res
layout_order_res


In [20]:
for layout in res_images[0].keys():
    print(res_images[0][layout])

<PIL.Image.Image image mode=RGB size=944x1363 at 0x7E0A387B48B0>
<PIL.Image.Image image mode=RGB size=944x1363 at 0x7E0A387B4940>


In [22]:
for layout in res_images[0].keys():
    layout_pdf = output_dir / f"{pdf_path.stem}_{layout}.pdf"
    pil_to_pdf_img2pdf([item[layout] for item in res_images], layout_pdf)

In [23]:
for item in markdown_images:
    if item:
        for path, image in item.items():
            file_path = output_dir / path
            file_path.parent.mkdir(parents=True, exist_ok=True)
            image.save(file_path)